In [1]:
import numpy as np
import pandas as pd
import glob
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tqdm

In [2]:
path = "/Users/user/Desktop/Oka//COMSOL/PEFC/data/24-7-18"
#path ="/Users/aruhy/OneDrive/デスクトップ/data/1-25"

In [17]:
df = pd.DataFrame(columns=['x座標','y座標','z座標','V[V]','Segment','i[A/m^2]'])

#CD_3D 110℃_0.4V_down
Segment_range = ["/*_up","/*_middle","/*_down"] 
for Segment in tqdm.tqdm(Segment_range):
        files2 = glob.glob(path + Segment)
        for filename in tqdm.tqdm(files2):
            
            df1 = pd.read_csv(filename,names=['x座標','y座標','z座標','i[A/m^2]'], delim_whitespace=True, header=8)
            v = re.findall(r'℃_(.*?)V_',filename)
            df1.insert(3,'V[V]', v[0])
            segment = re.findall(r'V_(.*?)$',filename)
            df1.insert(4,'Segment',segment[0] if segment else 'unknown')
            df = pd.concat([df, df1])
    
df['V[V]'] = df['V[V]'].astype(float)

mapping = {'up': 0.7,'middle': 1.4,'down': 2.1}  # 変換マッピングを定義
df['Segment'] = df['Segment'].map(mapping)  #変換を適用

df.reset_index(drop=True, inplace=True)
df

  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_7080\3574139763.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1])
100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


,x座標,y座標,z座標,V[V],Segment,i[A/m^2]
0,0.000785,0.0073,0.001180,0.25,0.7,934.026741
1,0.000831,0.0073,0.001180,0.25,0.7,934.046976
2,0.000000,0.0073,0.001175,0.25,0.7,933.568433
3,0.000000,0.0074,0.001175,0.25,0.7,933.547566
4,0.000000,0.0074,0.001177,0.25,0.7,933.542414
...,...,...,...,...,...,...
422389,0.000277,0.0150,0.001165,1.00,2.1,3.106792
422390,0.001708,0.0153,0.001165,1.00,2.1,3.104665
422391,0.000646,0.0150,0.001165,1.00,2.1,3.107525
422392,0.001431,0.0150,0.001168,1.00,2.1,3.106970


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422394 entries, 0 to 422393
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   x座標       422394 non-null  float64
 1   y座標       422394 non-null  float64
 2   z座標       422394 non-null  float64
 3   V[V]      422394 non-null  float64
 4   Segment   422394 non-null  float64
 5   i[A/m^2]  422394 non-null  float64
dtypes: float64(6)
memory usage: 19.3 MB


In [19]:
df.describe()

,x座標,y座標,z座標,V[V],Segment,i[A/m^2]
count,422394.000000,422394.000000,422394.000000,422394.000000,422394.000000,422394.000000
mean,0.000900,0.011109,0.001172,0.590842,1.392761,439.472979
std,0.000603,0.006534,0.000008,0.240724,0.570023,323.364935
min,0.000000,0.000000,0.001115,0.250000,0.700000,2.988277
25%,0.000369,0.005700,0.001165,0.350000,0.700000,128.690514
50%,0.000923,0.011000,0.001172,0.600000,1.400000,397.603445
75%,0.001431,0.016500,0.001180,0.800000,2.100000,776.003008
max,0.001800,0.022500,0.001230,1.000000,2.100000,934.893034


In [20]:
df.to_csv("CD3D_110℃_segment")

In [21]:
for col in df.columns:
    scaler =  StandardScaler()
    df[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1, 1))
    
df.head()

,x座標,y座標,z座標,V[V],Segment,i[A/m^2]
0,-0.191417,-0.582991,0.931613,-1.415906,-1.215323,1.529400
1,-0.114882,-0.582991,0.931613,-1.415906,-1.215323,1.529463
2,-1.492519,-0.582991,0.310611,-1.415906,-1.215323,1.527983
3,-1.492519,-0.567687,0.310611,-1.415906,-1.215323,1.527918
4,-1.492519,-0.567687,0.517611,-1.415906,-1.215323,1.527902


In [22]:
df.describe()

,x座標,y座標,z座標,V[V],Segment,i[A/m^2]
count,4.223940e+05,4.223940e+05,4.223940e+05,4.223940e+05,4.223940e+05,4.223940e+05
mean,-1.069866e-16,1.033531e-16,-2.185744e-14,1.937871e-16,1.797914e-15,2.583828e-16
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.492519e+00,-1.700217e+00,-7.141412e+00,-1.415906e+00,-1.215323e+00,-1.349822e+00
25%,-8.802358e-01,-8.278628e-01,-9.313931e-01,-1.000492e+00,-1.215323e+00,-9.610901e-01
50%,3.818925e-02,-1.672616e-02,-1.033905e-01,3.804278e-02,1.269905e-02,-1.294809e-01
75%,8.800789e-01,8.250194e-01,9.316126e-01,8.688707e-01,1.240722e+00,1.040714e+00
max,1.492362e+00,1.743287e+00,7.141632e+00,1.699699e+00,1.240722e+00,1.532079e+00


In [23]:
df.to_csv("CD3D_110℃_segment_std")